<a href="https://colab.research.google.com/github/RafaelGallo/Tensorflow-yolo-GPU/blob/main/CNN_Detectando_objetos_v%C3%ADdeos_YOLOv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CNN - Yolo Reconhecimento de objetos com Yolo e Opencv**

In [ ]:
from platform import python_version

print('Versão python neste Jupyter Notebook:', python_version())

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!pip install opencv-python==4.4.0.40

In [ ]:
import cv2
import numpy as np
import time
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt 
from google.colab.patches import cv2_imshow
import zipfile

In [ ]:
%reload_ext watermark
%watermark -a "Rafael Gallo" --iversions

In [ ]:
# Carregando os arquivos do modelo treinado

path =  '/content/gdrive/MyDrive/Colab Notebooks/Detecção de Objetos com YOLO, Darknet, OpenCV e Python - Udemy/modelo_YOLOv4.zip'
zip_object = zipfile.ZipFile(file=path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [ ]:
labels_path = os.path.sep.join(['/content/cfg', "coco.names"])
LABELS = open(labels_path).read().strip().split("\n")

weights_path = os.path.sep.join(['/content/', "yolov4.weights"])
config_path = os.path.sep.join(['/content/cfg', "yolov4.cfg"])

net = cv2.dnn.readNet(config_path, weights_path)

In [ ]:
# Definindo a detecção

np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")
ln = net.getLayerNames()

print("Todas as camadas (layers):")
print(ln)
print("Total: "+ str(len(ln)))
print("Camadas de saída: ")
print(net.getUnconnectedOutLayers())

ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

In [ ]:
# Criando funções detecção video

def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

# Construindo o blob da imagem

def blob_imagem(net, imagem, mostrar_texto=True):
  inicio = time.time() 
  blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)
  termino = time.time()
  if mostrar_texto:
    print("YOLO levou {:.2f} segundos".format(termino - inicio))
  return net, imagem, layerOutputs

#Realizando a detecção

def deteccoes(detection, _threshold, caixas, confiancas, IDclasses):
  scores = detection[5:] 
  classeID = np.argmax(scores)  
  confianca = scores[classeID]

  if confianca > _threshold:
      caixa = detection[0:4] * np.array([W, H, W, H])     
      (centerX, centerY, width, height) = caixa.astype("int")
            
      x = int(centerX - (width / 2))
      y = int(centerY - (height / 2))

      caixas.append([x, y, int(width), int(height)])
      confiancas.append(float(confianca))
      IDclasses.append(classeID)
      
  return caixas, confiancas, IDclasses

#Mostrando o resultado da detecção no video
def funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=True):  
  (x, y) = (caixas[i][0], caixas[i][1])
  (w, h) = (caixas[i][2], caixas[i][3])

  cor = [int(c) for c in COLORS[IDclasses[i]]]
  cv2.rectangle(imagem, (x, y), (x + w, y + h), cor, 2) 
  texto = "{}: {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])
  if mostrar_texto:
    print("> " + texto)
    print(x,y,w,h)
  cv2.putText(imagem, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor, 2)

  return imagem,x,y,w,h

In [ ]:
# Carregando vídeo para detecção

!cp /content/gdrive/MyDrive/Colab Notebooks/Detecção de Objetos com YOLO, Darknet, OpenCV e Python - Udemy/videos/NY_2.mp4

In [ ]:
# Lendo arquivo vídeo com o OpenCV

arquivo_video = '/content/gdrive/MyDrive/Colab Notebooks/Detecção de Objetos com YOLO, Darknet, OpenCV e Python - Udemy/videos/NY_2.mp4'
cap = cv2.VideoCapture(arquivo_video)
conectado, video = cap.read()
conectado

In [ ]:
video.shape

In [ ]:
video_largura = video.shape[1]
video_altura = video.shape[0]
video_largura, video_altura

In [ ]:
# Redimensionamento tamanho video

def redimensionar(largura, altura, largura_maxima = 600): 
  if (largura > largura_maxima):
    proporcao = largura / altura
    video_largura = largura_maxima
    video_altura = int(video_largura / proporcao)
  else:
    video_largura = largura
    video_altura = altura
  return video_largura, video_altura

video_largura, video_altura = redimensionar(video.shape[1], video.shape[0])
print(video_largura,video_altura)

In [ ]:
# Definindo configurações vídeo

nome_arquivo = 'resultado_1.avi' 
fourcc = cv2.VideoWriter_fourcc(*'XVID') 
fps = 24
saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

In [ ]:
# Definindo variáveis

threshold = 0.5
threshold_NMS = 0.3
fonte_pequena, fonte_media = 0.4, 0.6
fonte = cv2.FONT_HERSHEY_SIMPLEX
amostras_exibir = 20
amostra_atual = 0

In [ ]:
# Processamento do vídeo e exibição do resultado

while (cv2.waitKey(1) < 0):
  conectado, frame = cap.read()
  if not conectado:
    break
  t = time.time()
  frame = cv2.resize(frame, (video_largura, video_altura))
  try:
    (H, W) = frame.shape[:2]
  except:
    print('Erro')
    continue

  imagem_cp = frame.copy() 
  net, frame, layerOutputs = blob_imagem(net, frame)
  caixas = []       
  confiancas = []   
  IDclasses = []    

  for output in layerOutputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS)

  if len(objs) > 0:
    for i in objs.flatten():
      frame, x, y, w, h = funcoes_imagem(frame, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
      objeto = imagem_cp[y:y + h, x:x + w]
  
  cv2.putText(frame, " frame processado em {:.2f} segundos".format(time.time() - t), 
              (20, video_altura-20), fonte, fonte_pequena, (250, 250, 250), 0, lineType=cv2.LINE_AA)
  
  if amostra_atual <= amostras_exibir:
    cv2_imshow(frame)
    amostra_atual += 1

  saida_video.write(frame)

print('CNN Yolo - Terminou')
saida_video.release()
cv2.destroyAllWindows()

In [ ]:
# Download do video para o seu computador executando

!du -h resultado_1.avi
from google.colab import files

files.download("resultado_1.avi")